In [1]:
import polars as pl
from pathlib import Path

# Proje yolları
RAW_DATA_PATH = Path("../data/raw/MetroPT3(AirCompressor).csv")
PROCESSED_DATA_PATH = Path("../data/processed/metropt3_optimized.parquet")

# Lazy okuma — RAM'e tam yüklemez, sadece şemayı biraz okur kabaca bilgileri tahmin eder
df_lazy = pl.scan_csv(RAW_DATA_PATH)

# Şemayı gör — sütun isimleri ve mevcut veri tipleri
print(df_lazy.schema)

Schema({'': Int64, 'timestamp': String, 'TP2': Float64, 'TP3': Float64, 'H1': Float64, 'DV_pressure': Float64, 'Reservoirs': Float64, 'Oil_temperature': Float64, 'Motor_current': Float64, 'COMP': Float64, 'DV_eletric': Float64, 'Towers': Float64, 'MPG': Float64, 'LPS': Float64, 'Pressure_switch': Float64, 'Oil_level': Float64, 'Caudal_impulses': Float64})


/var/folders/mw/y5zpf1vn17bgfjkddgx020p00000gn/T/ipykernel_6847/1416601908.py:12: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  print(df_lazy.schema)


Pandas veriyi okurken boyutun 3-5 katı ram harcar ve tek çekirdek kullanır. polars ise lazy evaluation metodu ile akış halinde okur ekstra harcama yapmaz ve çok çekirdek kullandığında çok hızlıdır. (uyarının bi önemi yok sadece başlıkları görmek istiyorusan schema kullan diyor veriyi işlememe gerek kalmaz böylece diyor)

DÖNÜŞÜM : Ham CSV (data/raw/) → Polars Lazy Read (RAM'e tam yüklemez) → Downcasting (float64 → float32, int64 → int32) Ram kullanımını yarıya indirecek → Parquet Yaz (data/processed/)

In [2]:
# Şemayı doğru yöntemle al — uyarı olmadan
schema = df_lazy.collect_schema()
print(schema)

# Boş index sütununun adını görelim
print("\nSütun isimleri:", schema.names())

Schema({'': Int64, 'timestamp': String, 'TP2': Float64, 'TP3': Float64, 'H1': Float64, 'DV_pressure': Float64, 'Reservoirs': Float64, 'Oil_temperature': Float64, 'Motor_current': Float64, 'COMP': Float64, 'DV_eletric': Float64, 'Towers': Float64, 'MPG': Float64, 'LPS': Float64, 'Pressure_switch': Float64, 'Oil_level': Float64, 'Caudal_impulses': Float64})

Sütun isimleri: ['', 'timestamp', 'TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs', 'Oil_temperature', 'Motor_current', 'COMP', 'DV_eletric', 'Towers', 'MPG', 'LPS', 'Pressure_switch', 'Oil_level', 'Caudal_impulses']


Herşey normal sadece 64 lükleri 32 ye çeviricez, boş bir sütun var onu silicez ve zaman serisi string kalmış onu da gerçek datetime yapacağız

In [3]:
""" Sütunları optimize etme """
# Sensör sütunları — Float64'ten Float32'ye düşürülecekler
sensor_cols = [
    'TP2', 'TP3', 'H1', 'DV_pressure', 'Reservoirs',
    'Oil_temperature', 'Motor_current', 'COMP', 'DV_eletric',
    'Towers', 'MPG', 'LPS', 'Pressure_switch', 'Oil_level',
    'Caudal_impulses'
]

# Sütun Güncellemeleri
df_optimized = (
    df_lazy.drop('')                                          # Gereksiz index sütununu siler
    .with_columns(                #.with_columns ile yeni sütunlar ekleyebilir veya mevcut sütunları dönüştürebiliriz
        pl.col('timestamp').str.to_datetime())          # String sütunu Datetime dönüştürme 
    .with_columns(
        [pl.col(c).cast(pl.Float32) for c in sensor_cols])  # Float64 den Float32 dönüşümü
        # .col sütunu seçerken, .cast veri tipini değiştirirken kullanılır
)

# Parquet'e yazdırma
df_optimized.sink_parquet(PROCESSED_DATA_PATH) 
# veriyi parça parça rame yükle-işle-diske yaz (her parça için seri bu adımlar çalışır) adımlarını uygular
print("Parquet yazıldı")

Parquet yazıldı


csv metin tabanlıdır değerler karakter olarak saklanır, parquetde kolonsal saklama vardır sıkıştırma yapar buda csv nin 3 de biri kadar yer kaplatır ayrıca sonraki okumalarda da çok daha hızlı okur

In [4]:
""" CSV ve Parquet dosya boyutlarını karşılaştırma """
import os

csv_size = os.path.getsize("../data/raw/MetroPT3(AirCompressor).csv") / (1024 * 1024) 
parquet_size = os.path.getsize("../data/processed/metropt3_optimized.parquet") / (1024 * 1024)

print(f"CSV boyutu     : {csv_size:.1f} MB")
print(f"Parquet boyutu : {parquet_size:.1f} MB")
print(f"Küçülme oranı  : %{((csv_size - parquet_size) / csv_size * 100):.1f}")

CSV boyutu     : 208.2 MB
Parquet boyutu : 15.7 MB
Küçülme oranı  : %92.5


Bu aşırı iyi bir sonuç ancak temkinli olunmalı. Bu proje için bu oran normal kabul edilebilir

1- Saten csv den parquet e çevirdiğimizde %50 oranına ulaşırız. Örneğin csv 0.523841 gibi bir sayı aslında 7 karakter olarak saklanır yani 7 byte. Parquet aynı sayıyı binary olarak saklar Float32 için sadece 4 byte.

2- csv normal bildiğimiz satır satır saklıyor (zaman, tp2, tp3....) ancak parquete her özelliğin değerlerini kolon kolon saklıyor (tüm tp2 değerleri -> ..... diğer kolona tüm tp3 değerleri -> ...... gibi) tüm mesele de burda. parquete kolonlar ile sakladığından mesela bir  zaman içinde "TP2: 0.52, 0.52, 0.52, 0.52, 0.52, 0.52..." gibi 10bin kez tekrar eden bir değer geldiğinde parquete "Bu değer 10.000 kez tekrar ediyor, ben bunu bir kez yazarım yanına '10.000 kez' derim" ancak csv mecburen hepsini tek tek diğer özellikleriyle yazıyor. Bu olaya *Run-Length Encoding (RLE) denir

** Bizim verimizde makineler olduğundan  ve 2. seçenekteki durum çokça olduğundan (çünkü makineler çoğu zaman aynı durum içinde çalışır) ciddi bir tasarruf sağlıyoruz